In [74]:
from datasets import load_dataset
from datasets import load_metric
from transformers import AutoImageProcessor
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer
import torch
import numpy as np
from PIL import Image
import cv2
import os

c:\Users\Jayesh\AppData\Local\Programs\Python\Python39\lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [6]:
x_train = np.load('mhsma\\x_128_train.npy')
x_test = np.load('mhsma\\x_128_test.npy')
x_valid = np.load('mhsma\\x_128_valid.npy')
head_train_labels = np.load("mhsma\\y_head_train.npy")
head_test_labels = np.load("mhsma\\y_head_test.npy")
head_valid_labels = np.load("mhsma\\y_head_valid.npy")

In [8]:
def save_numpy_as_png(file_path, numpy_array):
    uint8_array = numpy_array.astype(np.uint8)
    image = Image.fromarray(uint8_array, 'L')
    image.save(file_path)

In [12]:
for img_array in x_train:
    print(img_array.shape)
    break

(128, 128)


In [15]:
for i, img_array in enumerate(x_valid):
    save_numpy_as_png(f"hf_dataset\\validation\\{str(head_valid_labels[i])}\\{str(i)}.png", img_array)

In [31]:
def convert_to_rgb(file_path):
    img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    cv2.imwrite(file_path, img)

In [46]:
files = os.listdir("hf_dataset\\validation\\1")

In [47]:
for file in files:
    convert_to_rgb(f"hf_dataset\\validation\\1\\{file}")

In [48]:
dataset = load_dataset("imagefolder", data_dir="hf_dataset")

Resolving data files: 100%|██████████| 300/300 [00:00<00:00, 149921.51it/s]
Extracting data files: 0it [00:00, ?it/s]
Extracting data files: 0it [00:00, ?it/s]
Extracting data files: 0it [00:00, ?it/s]
Generating train split: 1000 examples [00:00, 7073.98 examples/s]
Generating validation split: 240 examples [00:00, 9318.60 examples/s]
Generating test split: 300 examples [00:00, 9375.82 examples/s]


In [49]:
dataset["train"][0]

{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=128x128>,
 'label': 0}

In [50]:
id2label = {0 : "Normal", 1 : "Abnormal"}
label2id = {"Normal" : 0, "Abnormal" : 1}

In [51]:
model_checkpoint = "microsoft/swin-tiny-patch4-window7-224"
batch_size = 32

In [23]:
image_processor  = AutoImageProcessor.from_pretrained(model_checkpoint)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [24]:
image_processor

ViTImageProcessor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "resample": 3,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}

In [25]:
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    Resize,
    ToTensor,
)

c:\Users\Jayesh\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 'Could not find module 'C:\Users\Jayesh\AppData\Local\Programs\Python\Python39\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [57]:
normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
if "height" in image_processor.size:
    size = (image_processor.size["height"], image_processor.size["width"])
    crop_size = size
    max_size = None
elif "shortest_edge" in image_processor.size:
    size = image_processor.size["shortest_edge"]
    crop_size = (size, size)
    max_size = image_processor.size.get("longest_edge")

train_transforms = Compose(
        [
            RandomResizedCrop(crop_size),
            RandomHorizontalFlip(),
            ToTensor(),
            normalize,
        ]
    )

val_transforms = Compose(
        [
            Resize(size),
            CenterCrop(crop_size),
            ToTensor(),
            normalize,
        ]
    )

def preprocess_train(example_batch):
    """Apply train_transforms across a batch."""
    example_batch["pixel_values"] = [
        train_transforms(image.convert("RGB")) for image in example_batch["image"]
    ]
    return example_batch

def preprocess_val(example_batch):
    """Apply val_transforms across a batch."""
    example_batch["pixel_values"] = [val_transforms(image.convert("RGB")) for image in example_batch["image"]]
    return example_batch

In [58]:
train_ds = dataset["train"]
val_ds = dataset["validation"]

In [59]:
train_ds.set_transform(preprocess_train)
val_ds.set_transform(preprocess_val)

In [60]:
train_ds[0]

{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=128x128>,
 'label': 0,
 'pixel_values': tensor([[[-0.0972, -0.0972, -0.0972,  ...,  0.1426,  0.1426,  0.1426],
          [-0.0972, -0.0972, -0.0972,  ...,  0.1426,  0.1426,  0.1426],
          [-0.0972, -0.0972, -0.0972,  ...,  0.1426,  0.1426,  0.1426],
          ...,
          [ 0.1939,  0.1939,  0.1939,  ...,  0.0912,  0.0912,  0.0912],
          [ 0.1939,  0.1939,  0.1939,  ...,  0.0912,  0.0912,  0.0912],
          [ 0.1939,  0.1939,  0.1939,  ...,  0.0912,  0.0912,  0.0912]],
 
         [[ 0.0301,  0.0301,  0.0301,  ...,  0.2752,  0.2752,  0.2752],
          [ 0.0301,  0.0301,  0.0301,  ...,  0.2752,  0.2752,  0.2752],
          [ 0.0301,  0.0301,  0.0301,  ...,  0.2752,  0.2752,  0.2752],
          ...,
          [ 0.3277,  0.3277,  0.3277,  ...,  0.2227,  0.2227,  0.2227],
          [ 0.3277,  0.3277,  0.3277,  ...,  0.2227,  0.2227,  0.2227],
          [ 0.3277,  0.3277,  0.3277,  ...,  0.2227,  0.2227,  0.2227]],


In [75]:
model = AutoModelForImageClassification.from_pretrained(
    model_checkpoint, 
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes = True,
)

Some weights of SwinForImageClassification were not initialized from the model checkpoint at microsoft/swin-tiny-patch4-window7-224 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [76]:
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-eurosat",
    remove_unused_columns=False,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
)

In [80]:
metric = load_metric("accuracy")

In [77]:
def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [81]:
def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

In [82]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
)

LocalTokenNotFoundError: Token is required (`token=True`), but no token found. You need to provide a token or be logged in to Hugging Face with `huggingface-cli login` or `huggingface_hub.login`. See https://huggingface.co/settings/tokens.